In this Tutorial we are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- answering queries using this index,
- and organizing it as a simple web server.

Second part is devoted to spellchecking.

First, we need a unified approach to documents preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library or any other you know.

In [5]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        Ttext = word_tokenize(text)
        return Ttext

    
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        st = stemmer.stem(word)
        return st

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        val = False
        if word not in self.stop_words and word.isalpha():
          val = True
        return val

    
    def preprocess(self, text):
        #TODO combine all previous methods together: tokenize lowercased text 
        # and stem it, ignoring not appropriate words
        tokens = self.tokenize(text)
        ret = []
        for token in tokens:
          if self.is_apt_word(token.lower()):
            ret.append(self.stem(token, self.ps))
        return ret

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 1.1.1. Tests

In [134]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## 1.2. [25] Crawling and Indexing

### 1.2.1. [5] Base classes

Here are some base classes we will need for writing our indexer. The code from the first lab's solution is given, but note that you will need to change some of it, namely, the `parse` function (it is also possible to use your own implementation from the first homework, but make sure that it is correct). The reason is it always makes complete parsing, which we want to avoid when we only need links, for example, or a specific portion of text.

In [135]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlDocument(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self, check = False):
        #TODO change this method

        head = ""
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
            
        r = requests.get(self.url, allow_redirects=True) # Getting contents
        if r: 
          self.content = r.content

          model1 = BeautifulSoup(self.content)
          

          if "https://www.reuters.com/article" in self.url:
            header = model1.find("div", {"class": "ArticlePage-article-header-23J2O"})
            if header: 
              texts = header.h1.findAll(text=True)
              visible_texts = filter(tag_visible, texts)  
              head = u" ".join(t.strip() for t in visible_texts)

            model = model1.find("div", {"class": "ArticleBodyWrapper"})
          else:
            model = model1.find("div", {"id": "content"})

          
          if model:
            pass
          else:
            model = model1
          self.anchors = []
          a = model.find_all('a')
          for anchor in a:
              href = self.normalize(anchor.get('href'))
              text = anchor.text
              self.anchors.append((text, href))
          
          if check:
            self.images = []
            i = model.find_all('img')
            for img in i:
                href = self.normalize(img.get('src'))
                self.images.append(href)
          
          texts = model.findAll(text=True)
          visible_texts = filter(tag_visible, texts)  
          self.text = u" ".join(t.strip() for t in visible_texts)
          self.text =head + " " + self.text

### 1.2.2. [15] Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (of a form https://www.reuters.com/...) for visiting. To speed up, doesn't consider for visiting pages with content type other than html: '.pdf', '.mp3', '.avi', '.mp4', '.txt'. If encounters an article page (of a form https://www.reuters.com/article/...), saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`, `doc_id:url`
    - `index`, `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    - `doc_lengths`, `doc_id:doc_length` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.


**Extra task \* (no penalty to skip)** In real industrial systems a crawler would pass the links to the dedicated service that would load their contents in a bunch of parallel threads. Implement such a service - get urls as inputs, load page contents in parallel and return filenames on disk, which are then processed by indexer.

In [147]:
from collections import Counter
from queue import Queue
import pickle
import os

class Indexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
    
    def doc_checker(self, word, freq, doc_index):
      term_freq = 0.000
      doc = [freq]

      for val in doc_index:
        
        for item in doc_index[val]:
          
          if item[0] == word:
            
            term_freq = item[1]/freq
            doc.append((val, term_freq))
          
          else:
            
            term_freq = 0
      
      return doc
    
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        
        doc_urls = dict()
        doc_index = dict()
        doc_lengths = dict()
        inverted_index = dict()

        all_terms = []

        preprocessor = self.prep

        sequencedID = 0

        returnlist = []
        trunklist = []
        limitchecker = 1

        l = []
        l.append(source)

        for i in range(depth):
          
          smalllist = []        
          
          while (l !=[]):
            
            url = l.pop()
            
            if not "/video/" in url:
              
              trunklist.append(url)
              content = HtmlDocument(url)
              content.parse()

              try:
                preprocessed_text = self.index_doc(content.text, str(sequencedID))

                doc_urls[str(sequencedID)] = url

                doc_freq = Counter(preprocessed_text).items()

                doc_index[str(sequencedID)] = doc_freq

                all_terms.extend(preprocessed_text)

                sequencedID += 1            

                for (text, url) in content.anchors:
                  # url = c.url
                  if url:

                    if "https://www.reuters.com/" in url:  
                      if url not in trunklist and url not in smalllist and url not in l and (text, url) not in returnlist:
                        # print("inside:",url)
                        returnlist.append((text, url))
                        smalllist.append(url)
                        if "https://www.reuters.com/article" in url:
                          # print(url)
                          if limit and limitchecker>=limit:
                            yield content
                          limitchecker += 1
                          yield content
              except AttributeError:
                pass

                      
          l = smalllist
        
        all_terms_index = Counter(all_terms).items()
        
        for word in all_terms_index:
          l = self.doc_checker(word[0], word [1], doc_index)
          inverted_index[word[0]]= l
           

        self.doc_urls = doc_urls
        self.index = inverted_index
        # self.doc_lengths = doc_lengths
        # save results for later use
        
        if not os.path.exists(collection_path):
          os.makedirs(collection_path)

        with open(collection_path + '/doc_urls.p', 'wb') as fp:
            pickle.dump(self.doc_urls, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open(collection_path + '/inverted_index.p', 'wb') as fp:
            pickle.dump(self.index, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open(collection_path + '/doc_lengths.p', 'wb') as fp:
            pickle.dump(self.doc_lengths, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
        
    def index_doc(self, doc, doc_id):
        #TODO add documents to index

        preprocessed_text = self.prep.preprocess(doc)
        self.doc_lengths[doc_id] = preprocessed_text

        return preprocessed_text

### 1.2.3. Tests

In [148]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/technology", 2, "test_collection", 5):
    print(k, c.url)
    k+=1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/technology
2 https://www.reuters.com/technology
3 https://www.reuters.com/technology
4 https://www.reuters.com/technology
5 https://www.reuters.com/technology
6 https://www.reuters.com/technology
7 https://www.reuters.com/technology
8 https://www.reuters.com/technology
9 https://www.reuters.com/technology
10 https://www.reuters.com/technology
11 https://www.reuters.com/technology
12 https://www.reuters.com/technology
13 https://www.reuters.com/technology
14 https://www.reuters.com/technology
15 https://www.reuters.com/technology
16 https://www.reuters.com/technology
17 https://www.reuters.com/news/archive/technologynews?view=page&page=2&pageSize=10
18 https://www.reuters.com/news/archive/technologynews?view=page&page=2&pageSize=10
19 https://www.reuters.com/news/archive/technologynews?view=page&page=2&pageSize=10
20 https://www.reuters.com/news/archive/technologynews?view=page&page=2&pageSize=10
21 https://www.reuters.com/news/archive/technologynews?view=page&

### 1.2.4. Building an index

In [149]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/", 3, "docs_collection"):
    print(k, c.url)
    k+=1

1 https://www.reuters.com/
2 https://www.reuters.com/
3 https://www.reuters.com/
4 https://www.reuters.com/
5 https://www.reuters.com/
6 https://www.reuters.com/
7 https://www.reuters.com/
8 https://www.reuters.com/
9 https://www.reuters.com/
10 https://www.reuters.com/
11 https://www.reuters.com/
12 https://www.reuters.com/
13 https://www.reuters.com/
14 https://www.reuters.com/
15 https://www.reuters.com/
16 https://www.reuters.com/
17 https://www.reuters.com/
18 https://www.reuters.com/
19 https://www.reuters.com/
20 https://www.reuters.com/
21 https://www.reuters.com/
22 https://www.reuters.com/
23 https://www.reuters.com/
24 https://www.reuters.com/
25 https://www.reuters.com/
26 https://www.reuters.com/
27 https://www.reuters.com/
28 https://www.reuters.com/
29 https://www.reuters.com/
30 https://www.reuters.com/
31 https://www.reuters.com/
32 https://www.reuters.com/
33 https://www.reuters.com/
34 https://www.reuters.com/
35 https://www.reuters.com/
36 https://www.reuters.com/
3

### 1.2.5. [5] Index statistics

Load an index and print the statistics.

In [150]:
# load index, doc_lengths and doc_urls
with open('inverted_index.p', 'rb') as fp:
    index = pickle.load(fp)
with open('doc_lengths.p', 'rb') as fp:
    doc_lengths = pickle.load(fp)
with open('doc_urls.p', 'rb') as fp:
    doc_urls = pickle.load(fp)
    
    
print('Total index length', len(index))
print('\nTop terms by number of documents they apperared in:')
sorted_by_n_docs = sorted(index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])
print('\nTop terms by overall frequency:')
sorted_by_freq = sorted(index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index length 8527

Top terms by number of documents they apperared in:
[('reuter', 242), ('said', 233), ('our', 205), ('thomson', 196), ('read', 195), ('report', 194), ('min', 194), ('after', 191), ('new', 186), ('trust', 182), ('standard', 181), ('principl', 181), ('s', 180), ('year', 178), ('thi', 169), ('more', 169), ('have', 169), ('who', 167), ('edit', 167), ('last', 160)]

Top terms by overall frequency:
[('reuter', 1168), ('said', 961), ('s', 828), ('februari', 827), ('after', 749), ('new', 735), ('close', 551), ('hi', 528), ('more', 523), ('year', 458), ('have', 450), ('who', 423), ('super', 412), ('friday', 411), ('bowl', 403), ('report', 400), ('dure', 391), ('not', 389), ('feb', 379), ('thi', 368)]


## 1.3. [15] Answering query

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [151]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        #TODO retrieve a set of documents containing all query terms
        
        prob = [0, 0, 0, 0, 0, 0]
        for q in query:
          for i in index:
            for k in index[i][1:]:
              if q == i:
                prob[k[0]] += 1
        max_index = prob.index(max(prob))

        return {max_index}

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores

        N = len(doc_lengths)

        avgdl = 0.0
        sum = 0
        
        for doc in doc_lengths:
          sum += doc_lengths[doc]
        
        avgdl =sum/N
        
        n={}
        prob = [0, 0, 0, 0, 0, 0]
        for q in query:
          p=0
          for i in index:
            if q == i:
              n[q] = index[i][0]
        

        scores = dict()

        for i in index:
                    
          for sheedaDaku in index[i][1:]:
            score = 0.0
            tf = 0.0
            for q in query:

              if q == i:
                freq = index[i][0]
                # print(in)

                docID = sheedaDaku[0]

                dtf = sheedaDaku[1]
                
                tf = dtf / freq

                length = doc_lengths[docID]
                
                idf = math.log10(1 + ((N-n[q]+0.5)/(n[q]+0.5)))
                
                score = (idf * tf * (k1 +1))/(tf + (k1 *(1 - b + (b*(length/avgdl)))))

                if docID in scores:
                  scores[docID] += score
                else:
                  scores[docID] = score

                # score += (idf * tf * (k1 +1))/(tf + (k1 *(1 - b + (b*(length/avgdl)))))
              # print(docID, idf, score, q)

            # scores[docID] += score

        return scores

### 1.3.1. Tests

In [152]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

## 1.4. Setting up a server

**Extra task \* (no penaly if skipped)** Organize the resulting search engine as a web-service that gets a query from get-parameters and returns urls with scores as a `json` dictionary. Check its work in a browser of with curl, should look smth like this:
 
`> curl localhost:8080/?q=some_query_text
{ "url1" : 0.9, "url2": 0.8 }`

You can use one of the following tools for this task: 
- https://www.acmesystems.it/python_http, 
- [http.server.ThreadingHTTPServer (3.7+)](https://docs.python.org/3/library/http.server.html#http.server.SimpleHTTPRequestHandler)
- [Flask](https://pypi.org/project/Flask/)

In [153]:
#TODO write a web-service that answers queries using inverted index

# 2. [55] Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. User knows that he doesn't know the correct spelling OR he wants to get the results that follow some known pattern, so he uses so called wildcards - queries like `retr*val`;
2. User doesn't know the correct spelling OR he doesn't care OR he's in a rush OR he expects his mistakes will be corrected OR your option, so he makes mistakes and we need to handle them using:

    2.1. Simple spellchecker by Peter Norvig;
    
    2.2. Phonetic correction by means of Soundex algorithm;
    
    2.3. Trigrams with Jaccard coefficient.

## 2.1. [20] Handling wildcards

We will handle wildcard queries using k-grams. K-grams is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 2.1.1. Downloading the dataset

In [1]:
import urllib.request
data_url = "https://raw.githubusercontent.com/hsu-ai-course/hsu.ai/master/code/datasets/nlp/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 2.1.2. [20] Implementation of search

In [128]:
import re
from collections import Counter

def doc_checker(word, freq, doc_index):
  term_freq = 0.000
  doc = [freq]

  for val in doc_index:
    
    for item in doc_index[val]:
      
      if item[0] == word:
        
        term_freq = item[1]/freq
        doc.append((val, term_freq))
      
      else:
        
        term_freq = 0
  
  return doc

def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)
    prep = Preprocessor()
    inverted_index = {}
    doc_index = dict()

    all_text = []
    sequencedID = 0

    for fact in documents:
      # print(fact)

      preprocessed_text = []

      tokens = prep.tokenize(fact)
      for token in tokens:
        if prep.is_apt_word(token):
          preprocessed_text.append(token)
      doc_freq = Counter(preprocessed_text).items()

      doc_index[str(sequencedID)] = doc_freq
      all_text.extend(preprocessed_text)
      sequencedID += 1

    all_terms_index = Counter(all_text).items()
        
    for word in all_terms_index:
      l = doc_checker(word[0], word [1], doc_index)
      inverted_index[word[0]]= l
            
    return inverted_index


def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Padd with '$' ($word$) before splitting to k-grams    
    k_gram_index = {}

    def ngram(text,grams):  
      model=[]
      # model will contain n-gram strings
      count=0
      for token in text[:len(text)-grams+1]:  
        model.append(text[count:count+grams])  
        count=count+1  
      return model

    for term in inverted_index:
      k_gram_index[term] = ngram('$'+ term+'$', k)
    
    return k_gram_index

def isStar(wcard, grams):
    model=[]
    # model will contain n-gram strings
    wcard = wcard.split("*")
    for w in wcard:
      count=0
      if len(w)<grams:
        model.append(w)
      else: 
        for token in w[:len(w)-grams+1]:
          model.append(w[count:count+grams])  
          count=count+1 
    return model


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    #TODO for a given wildcard return all words matching it using k-grams
    # refer to book chapter 3.2.2
    # don't forget to pad wildcard with '$', when appropriate
    
    def ngram(text,grams):  
      model=[]
      # model will contain n-gram strings
      count=0
      for token in text[:len(text)-grams+1]:  
        model.append(text[count:count+grams])  
        count=count+1  
      return model
    if "*" in wildcard:
      wildcard_invertedIndex = isStar('$'+ wildcard +'$', 3)
    else:
      wildcard_invertedIndex = ngram('$'+ wildcard +'$', 3)
      
    returnlist = []
    r = []
    for wildcrd in wildcard_invertedIndex:
      for k_gram in k_gram_index:
        if wildcrd in k_gram_index[k_gram]:
          if k_gram not in returnlist:
            returnlist.append(k_gram)
          else:  
            r.append(k_gram)
    
    return r


def search_wildcard(wildcard, k_gram_index, index, docs):
    #TODO retrive list of documnets (facts) that contain words matching wildcard
    returnlist = []
    matching_words = generate_wildcard_options(wildcard, k_gram_index, index)
    for word in matching_words:
      # print(word)
      for fact in facts:
        if word[0] in fact:
          returnlist.append(fact)
    return returnlist

### 2.1.3. Tests

In [130]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['red', 'recorded', 'received', 'reduced']
1. If you somehow found a way to extract all of the gold from the bubbling core of our lovely little planet, you would be able to cover all of the land in a layer of gold up to your knees.
2. McDonalds calls frequent buyers of their food "heavy users."
3. The average person spends 6 months of their lifetime waiting on a red light to turn green.
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
5. You burn more calories sleeping than you do watching television.
6. There are more lifeforms living on your skin than there are people on the planet.
7. Southern sea otters have flaps of skin under their forelegs that act as pockets. When diving, they use these pouches to store rocks and food.
8. In 1386 a pig in France was executed by public hanging for the murder of a child.
9. One in every five adults believe that aliens are hiding in our planet disguised as humans.
10. If you believe that you're truly one

## 2.2. [35] Handling typos

### 2.2.1 Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [84]:
!pip install jsonlines

In [85]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [88]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

### 2.2.2. [5] Build a dataset vocabulary
We will need it for Norvig's spellchecker as well as for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [89]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [90]:
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

58392

In [91]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6100), ('de', 80), ('deutsch', 4), ('nocomments', 2), ('you', 41999), ('can', 26004), ('disable', 527), ('comments', 351), ('for', 44674), ('the', 206912)]


### 2.2.3. [25] Implement context-independent spellcheker ##

0) Write code to compute editorial distance

1) [Norvig's corrector](https://norvig.com/spell-correct.html)

2) [Soundex](https://en.wikipedia.org/wiki/Soundex)

3) Trigrams with Jaccard coefficient.

#### 2.2.3.0. [5] Editorial distance

Frequently used distance measure between two character sequences. We will use this distance to sort Soundex search results.

In [135]:
# cite - https://www.geeksforgeeks.org/edit-distance-dp-5/
def edit_dist(s1, s2) -> int:
    # TODO compute the Damerau-Levenshtein distance between two given strings (s1 and s2)

    str1 = s1
    str2 = s2
    len1 = len(str1)
    len2 = len(str2)
    
    dist = 1

    if sorted(str1) == sorted(str2):
      for k in range(len1):
        if not str1[k] == str2[k]:
          dist += 1
    
    if not dist ==1:
      dist -= 1
    
    DP = [[0 for i in range(len1 + 1)] 
             for j in range(2)];
    
    for i in range(0, len1 + 1):
        DP[0][i] = i
    
    for i in range(1, len2 + 1):
        for j in range(0, len1 + 1):
            if (j == 0):
                DP[i % 2][j] = i
            elif(str1[j - 1] == str2[i-1]):
                DP[i % 2][j] = DP[(i - 1) % 2][j - 1]
             
            else:
                DP[i % 2][j] = (1 + min(DP[(i - 1) % 2][j], 
                                    min(DP[i % 2][j - 1], 
                                  DP[(i - 1) % 2][j - 1])))
             
    distance = DP[len2 % 2][len1]/dist
    return int(distance)

In [136]:
# tests

assert edit_dist("korrectud", "corrected") == 2, "Edit distance is computed incorrectly"
assert edit_dist("soem", "some") == 1, "Edit distance is computed incorrectly"
assert edit_dist("one", "one") == 0, "Edit distance is computed incorrectly"

#### 2.2.3.1. [5] Norvig's spellchecker

In [275]:
import re
from collections import Counter
# cite https://norvig.com/spell-correct.html
WORDS = vocabulary

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def fix_typo_norvig(word) -> str:
    #TODO return best matching result for the word
    value = correction(word) 
    if value:
      return value
    return word

In [276]:
# tests

assert fix_typo_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_norvig("speling") == "spelling", "Norvig's correcter doesn't work"

#### 2.2.3.2. [10] Soundex 

For cases when the exact spelling is unknown, phonetic algorithms such as Soundex can be very helpful - they allow user to type a word the way he thinks it should sound, and then suggest the corrrect version. Go through *chapter 3.4* to understand how Soundex algorithm works.

In [277]:
def produce_soundex_code(word):
    #TODO implement Soundex algorithm, version from book chapter 3.4
    # input word is already lowercased
    # return Soundex 4-character code, like 'k450'
    # cite - https://www.codedrome.com/the-soundex-algorithm-in-python/

    name = word
    soundexcoding = [' ', ' ', ' ', ' ']
    soundexcodingindex = 1

    #           ABCDEFGHIJKLMNOPQRSTUVWXYZ
    mappings = "01230120022455012623010202"

    soundexcoding[0] = name[0].upper()

    for i in range(1, len(name)):

         try :
           c = ord(name[i].upper()) - 65
         except:
           return ""


         if c >= 0 and c <= 25:

             if mappings[c] != '0':

                 if mappings[c] != soundexcoding[soundexcodingindex-1]:

                     soundexcoding[soundexcodingindex] = mappings[c]
                     soundexcodingindex += 1

                 if soundexcodingindex > 3:

                     break

    if soundexcodingindex <= 3:
        while(soundexcodingindex <= 3):
            soundexcoding[soundexcodingindex] = '0'
            soundexcodingindex += 1

    return ''.join(soundexcoding)
    # return ""


def build_soundex_index(dictionary):
    #TODO build soundex index for dictionary words.
    # dictionary is a vocabulary of original words
    # output format: 'code1': ['word1_with_code1', 'word2_with_code1', ...]    
    soundex_index = {}

    for word in dictionary:
      code = produce_soundex_code(word)
      if code in soundex_index:
        soundex_index[code].append(word)
      else:
        soundex_index[code] = [word]
    
    return soundex_index


def fix_typo_soundex(word, soundex_index) -> list:
    #TODO return words from vocabulary that match with result by soundex fingerprint
    # ordered results by editorial distance
    result=[]
    code = produce_soundex_code(word) 
    
    if code in soundex_index:
      return build_soundex_index(vocabulary)[code]

    return word

In [278]:
# tests

soundex_index = build_soundex_index(vocabulary)

code1 = produce_soundex_code("britney")
code2 = produce_soundex_code("breatany")
print(code1, code2)
assert code1 == code2

print(fix_typo_soundex("enouhg", soundex_index))
assert "enough" in fix_typo_soundex("enouhg", soundex_index), "Assert soundex failed"

B635 B635
['enjoy', 'enough', 'emacs', 'enhance', 'enums', 'emojis', 'emoji', 'enhances', 'emc', 'engage', 'enqueues', 'emas', 'enqueue', 'euank', 'enjoys', 'enosys', 'ensue', 'enemies', 'enmasse', 'emac', 'ens', 'enc', 'eng', 'emojii', 'enki', 'enso', 'ensues', 'enzo', 'engages', 'enwiki', 'emmc', 'emesh', 'emsize', 'emg', 'emojies', 'emgo']


#### 2.2.3.3. [5] Trigrams with Jaccard coefficient

In [279]:
def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index

    def caljaccardCoefficient(list1, list2):
      intersection = len(list(set(list1).intersection(list2)))
      union = (len(list1) + len(list2)) - intersection
      return float(intersection) / union

    def ngram(text,grams):  
      model=[]
      # model will contain n-gram strings
      count=0
      for token in text[:len(text)-grams+1]:  
        model.append(text[count:count+grams])  
        count=count+1  
      return model
    
    distances = []
    word_gram = ngram('$'+ word +'$', 3)
    i = 0
    for g in word_gram:      
      for k in k_gram_index:
        for kg in k_gram_index[k]:  
          if kg == g:
            distances.append(k)
    
    
    filteredList = Counter(distances).items()
    newList = []
    for f in filteredList:
      if f[1]>1:
        newList.append(f[0])
        # print(f[0])
    
    returnList = []
    for val in newList:
      score = caljaccardCoefficient(word, val)
      if score > 0.4:
        returnList.append(val)
    
    if returnList == []:
      return [word]

    return returnList

In [280]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'enormous', 'eno', 'enought', 'renounce', 'homogenous', 'heterogenous', 'hetrogenous']


## 2.2.4. [5] Estimate quality

In [ ]:
norvig, soundex, kgram = 0, 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, ss, sk = src.lower(), src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, ws, wk = fix_typo_norvig(word), \
                         fix_typo_soundex(word, soundex_index)[0], \
                         fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            ss = ss.replace(word, ws)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    soundex += int(ss == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"Soundex accuracy ({soundex}) = {soundex / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")